In [9]:
# Scroll to the very bottom to view the data calculations for both TOA and Temperature Deviation
import numpy as np
import pandas as pd 
from netCDF4 import Dataset 
import numpy.ma as ma 
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

filename = "CERES_SYN1.nc"
cwDir = "/Users/kristenbrown/Desktop/SciViz/project/"
ds = Dataset(cwDir+filename, mode="r")
print(ds)



<type 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format UNDEFINED):
    title: CERES SYN1deg Products - Monthly Means
    institution: NASA/LaRC (Langley Research Center) Hampton, Va
    Conventions: CF-1.4
    comment: Data is from East to West and South to North. See values in latitude and longitudes dimensions.
    version: This is version 3A: May 2, 2014
    Fill_Value: Fill Value is -999.0
    dimensions(sizes): lon(360), lat(180), time(182)
    variables(dimensions): float32 lon(lon), float32 lat(lat), int32 time(time), float32 toa_net_all_mon(time,lat,lon), float32 toa_net_clr_mon(time,lat,lon), float32 aux_skint_mon(time,lat,lon)
    groups: 



In [3]:
#pull out relevant data
time = ds.variables['time'][:]
lat = ds.variables['lat'][:]
lon = ds.variables['lon'][:]
toa = ds.variables['toa_net_all_mon'][:,:,:]
skin = ds.variables['aux_skint_mon'][:,:,:]

# this part I was just checking the attributes for each variable so I could save it all the same again later. 
print(ds.variables['time']) 
print time[0]

<type 'netCDF4._netCDF4.Variable'>
int32 time(time)
    long_name: time
    units: days since 2000-03-01 00:00:00
    delta_t: 0000-00-01 00:00:00
unlimited dimensions: 
current shape = (182,)
filling off

14


In [4]:
#calculate the average over time at each lat/lon position
toaAvgLL = np.mean(toa[:,:,:],0)
skinAvgLL = np.mean(skin[:,:,:],0)

#calculate the deviation from average
toa_loc = toa - toaAvgLL
skin_loc = skin - skinAvgLL

#what is the min/max (for color table purposes it's good to know)
print "TOA min deviation: ", ma.min(toa_loc), "\nTOA max deviation: ", ma.max(toa_loc)
print "\nSkin temp min deviation: ", ma.min(skin_loc), "\nSkin temp max deviation: ", ma.max(skin_loc)

TOA min deviation:  -182.563 
TOA max deviation:  261.303

Skin temp min deviation:  -37.198 
Skin temp max deviation:  40.537


In [5]:
#calculate global average 
toaAvg = ma.average(toa)
skinAvg = ma.average(skin)
print "Global average TOA: ", toaAvg, "\nGlobal average Skin Temp: ", skinAvg

#calculate difference from global average
toa_glob = toa - toaAvg
skin_glob = skin - skinAvg

#what is the min/max (for color table purposes it's good to know)
print "\nTOA min deviation: ", ma.min(toa_glob), "\nTOA max deviation: ", ma.max(toa_glob)
print "\nSkin temp min deviation: ", ma.min(skin_glob), "\nSkin temp max deviation: ", ma.max(skin_glob)

Global average TOA:  -25.9306362773 
Global average Skin Temp:  278.866281373

TOA min deviation:  -188.763 
TOA max deviation:  222.131

Skin temp min deviation:  -84.5329 
Skin temp max deviation:  40.3689


In [6]:
#pull out which indices would correspond to the same month in our dataset
months=np.array([0,1,2,3,4,5,6,7,8,9,10,11])
all_months=np.array([0,1,2,3,4,5,6,7,8,9,10,11])

for i in range(1,15): #each month occurs 15 times except for two months extra
    months+=12
    all_months=np.vstack((all_months, months))

all_months = np.transpose(all_months) #so that it is divided properly

print(all_months) #check that it works, just missing the last two months which I add back in later

[[  0  12  24  36  48  60  72  84  96 108 120 132 144 156 168]
 [  1  13  25  37  49  61  73  85  97 109 121 133 145 157 169]
 [  2  14  26  38  50  62  74  86  98 110 122 134 146 158 170]
 [  3  15  27  39  51  63  75  87  99 111 123 135 147 159 171]
 [  4  16  28  40  52  64  76  88 100 112 124 136 148 160 172]
 [  5  17  29  41  53  65  77  89 101 113 125 137 149 161 173]
 [  6  18  30  42  54  66  78  90 102 114 126 138 150 162 174]
 [  7  19  31  43  55  67  79  91 103 115 127 139 151 163 175]
 [  8  20  32  44  56  68  80  92 104 116 128 140 152 164 176]
 [  9  21  33  45  57  69  81  93 105 117 129 141 153 165 177]
 [ 10  22  34  46  58  70  82  94 106 118 130 142 154 166 178]
 [ 11  23  35  47  59  71  83  95 107 119 131 143 155 167 179]]


In [7]:
#initialize variables in same shape as toa/skin arrays. 
toa_m_loc = np.empty([182,180,360])
toa_m_glob = np.empty([182,180,360])
skin_m_loc = np.empty([182,180,360])
skin_m_glob = np.empty([182,180,360])

#calculate monthly averages and deviations from those averages
for i in range(1,13):
    #just getting the right months set up
    if i is 1:
        submonths = np.concatenate((all_months[i-1],[180])) #need to add on the extra month
    elif i is 2: 
        submonths = np.concatenate((all_months[i-1],[181])) #need to add on the extra month       
    else:
        submonths = all_months[i-1] #all others were just 15 years exactly
    
    #actual calculation for each month, stored in the same places as original data
    toa_m_loc[submonths,:,:] = toa[submonths,:,:] - np.mean(toa[submonths,:,:],0)
    toa_m_glob[submonths,:,:] = toa[submonths,:,:] - ma.average(toa[submonths,:,:])
    skin_m_loc[submonths,:,:] = skin[submonths,:,:] - np.mean(skin[submonths,:,:],0)
    skin_m_glob[submonths,:,:] = skin[submonths,:,:] - ma.average(skin[submonths,:,:])

#what is the min/max (for color table purposes it's good to know)
print "Deviation from global averages by month...."
print "\nTOA min deviation: ", ma.min(toa_m_glob), "\nTOA max deviation: ", ma.max(toa_m_glob)
print "\nSkin temp min deviation: ", ma.min(skin_m_glob), "\nSkin temp max deviation: ", ma.max(skin_m_glob) 


#what is the min/max (for color table purposes it's good to know)
print "Deviation from averages by month and lat/lon...."
print "\nTOA min deviation: ", ma.min(toa_m_loc), "\nTOA max deviation: ", ma.max(toa_m_loc)
print "\nSkin temp min deviation: ", ma.min(skin_m_loc), "\nSkin temp max deviation: ", ma.max(skin_m_loc)


Deviation from global averages by month....

TOA min deviation:  -194.428390503 
TOA max deviation:  213.714996338

Skin temp min deviation:  -86.9217529297 
Skin temp max deviation:  39.5655517578
Deviation from averages by month and lat/lon....

TOA min deviation:  -189.832275391 
TOA max deviation:  123.84236145

Skin temp min deviation:  -24.3057556152 
Skin temp max deviation:  22.6296234131


In [8]:
# decided to add one more variable. This variable uses the first year (Jan 2000- Dec 2000) as a baseline. 
all_years = np.transpose(all_months)
skin_base = np.empty([182,180,360])
toa_base = np.empty([182,180,360])

for i in range(1,16):
    skin_base[all_years[i-1],:,:] = skin[all_years[i-1],:,:] - np.mean(skin[[0,1,2,3,4,5,6,7,8,9,10,11],:,:],0)
    toa_base[all_years[i-1],:,:] = toa[all_years[i-1],:,:] - np.mean(toa[[0,1,2,3,4,5,6,7,8,9,10,11],:,:],0)

skin_base[[180,181],:,:] = skin[[180,181],:,:] - np.mean(skin[[0,1,2,3,4,5,6,7,8,9,10,11],:,:],0)
toa_base[[180,181],:,:] = toa[[180,181],:,:] - np.mean(skin[[0,1,2,3,4,5,6,7,8,9,10,11],:,:],0)


#what is the min/max (for color table purposes it's good to know)
print "Deviation from averages by month and lat/lon...."
print "\nTOA min deviation: ", ma.min(toa_base), "\nTOA max deviation: ", ma.max(toa_base)
print "\nSkin temp min deviation: ", ma.min(skin_base), "\nSkin temp max deviation: ", ma.max(skin_base)

#for the line plots:
print "\nDeviation from temperature in 2000....\n"

for i in range(1,16):
    print "Year "+str(i-1)+": ", ma.average(skin[all_years[i-1],:,:]) - ma.average(skin[[all_years[0]],:,:])
    
print "\nDeviation of TOA...\n"
for l in range(1,16):
    print "Year "+str(l-1)+": ", ma.average(toa[all_years[l-1],:,:]) - ma.average(toa[[all_years[0]],:,:])    

Deviation from averages by month and lat/lon....

TOA min deviation:  -440.646484375 
TOA max deviation:  272.119720459

Skin temp min deviation:  -37.392868042 
Skin temp max deviation:  43.5535736084

Deviation from temperature in 2000....

Year 0:  0.0
Year 1:  0.133909465021
Year 2:  0.349773662551
Year 3:  0.198353909465
Year 4:  0.149691358025
Year 5:  0.454691358025
Year 6:  0.279670781893
Year 7:  0.280267489712
Year 8:  0.220185185185
Year 9:  0.218683127572
Year 10:  0.131255144033
Year 11:  0.219444444444
Year 12:  0.258230452675
Year 13:  0.318065843621
Year 14:  0.377757201646

Deviation of TOA...

Year 0:  0.0
Year 1:  -0.192646604938
Year 2:  -0.430643004115
Year 3:  0.306247427984
Year 4:  0.268840020576
Year 5:  0.0950077160494
Year 6:  0.388973765432
Year 7:  0.122947530864
Year 8:  0.904308127572
Year 9:  0.278513374486
Year 10:  0.14120627572
Year 11:  0.50829218107
Year 12:  0.483518518519
Year 13:  0.00198816872428
Year 14:  0.149606481481
